<a href="https://colab.research.google.com/github/Haruka0522/Small-Programs/blob/master/practice_gan/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.layers import Activation, BatchNormalization, Concatenate, Dense, Embedding, Flatten, Input, Multiply, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Model, Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


モデルを定義

In [4]:
#入力
img_row = 28
img_col = 28
channels = 1

img_shape = (img_row,img_col,channels)

z_dim = 100
num_classes = 10  #データセットのクラス数

In [5]:
#Gモデル
def build_generator(z_dim):
  model = Sequential()
  model.add(Dense(256,input_shape=(z_dim,),activation="relu"))
  model.add(Dense(512, activation="relu"))
  model.add(Dense(1024, activation="relu"))
  model.add(Dense(np.prod(img_shape),activation="tanh"))

  model.add(Reshape(img_shape))

  model.summary()
  return model

In [6]:
#Dモデル
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(256,activation="relu"))
    model.add(Dense(1,activation="relu"))
    model.summary()
    return model

In [7]:
#DとGの合併
def build_combined1():
    discriminator.trainable = False
    model = Sequential([generator,discriminator])
    return model

In [8]:
optimizer = Adam(0.0002,0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
#Dモデル構築
discriminator = build_discriminator(img_shape)
discriminator.compile(loss="binary_crossentropy",
                      optimizer=optimizer,
                      metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [10]:
#Gモデル構築
generator = build_generator(z_dim)
combined = build_combined1()
combined.compile(loss="binary_crossentropy",
                 optimizer=optimizer)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_7 (Dense)              (None, 784)               803600    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
Total params: 1,486,352
Trainable params: 1,486,352
Non-trainable params: 0
_________________________________________________________________


学習部分

In [15]:
def train(epochs,batch_size,save_interval):
    (X_train,_),(_,_) = mnist.load_data()

    #正規化
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train,axis=3)

    half_batch = int(batch_size / 2)

    for epoch in range(epochs):
        save_imgs(epoch)
        for iteration in range(batch_size):
            noise = np.random.normal(0, 1, (half_batch, z_dim))
            gen_imgs = generator.predict(noise)

            idx = np.random.randint(0, X_train.shape[0],half_batch)
            imgs = X_train[idx]

            #Dの学習
            d_loss_real = discriminator.train_on_batch(imgs,
                                                       np.ones((half_batch,1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs,
                                                       np.zeros((half_batch,1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #Gの学習
            noise = np.random.normal(0,1,(batch_size,z_dim))
            valid_y = np.array([1] * batch_size)
            g_loss = combined.train_on_batch(noise,valid_y)


保存

In [16]:
def save_imgs(epoch):
    r,c = 5,5

    noise = np.random.normal(0,1,(r*c,z_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig,axs = plt.subplots(r,c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap="gray")
            axs[i,j].axis("off")
            cnt += 1
    fig.savefig(f"mnist_{epoch}.png")
    plt.close()

実行

In [17]:
import time

t1 = time.time()
train(epochs=100,batch_size=100,save_interval=5)
t2 = time.time()

elapsed_time = t2 - t1
print(f"経過時間：{elapsed_time}")

/tensorflow-1.15.2/python3.6/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


経過時間：175.86712408065796
